<a href="https://colab.research.google.com/github/victor1cg/Python/blob/master/Confitec_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# Avaliação Confitec #**

##### Nome : Victor Goncalves
##### Data : 13/03/2022
#### -------- PySpark --------

In [81]:
# Utilizo o Google Colab para rodar o script e sessão de execução
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [82]:
# configurar as variáveis de ambiente JAVA e SPARK
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [83]:
import pyspark
import pyspark.sql.functions as func

from pyspark.sql.types import *
from pyspark.sql import SparkSession
from datetime import *
from datetime import datetime

spark = SparkSession.builder.getOrCreate()

print(spark)

In [133]:
# Importando os dados .parquet ao dataframe spark
df = spark.read.parquet('OriginaisNetflix.parquet')

In [104]:
# Analise dos dados importado :
# > 358 linhas e 15 colunas
print(df.count())
print(len(df.columns))

# visualizar dados Top 10
df.show(10)
# visualizar Schema
df.printSchema()

358
15
+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+-------+------+-----+--------+--------------------+
|               Title|               Genre|         GenreLabels| Premiere|             Seasons|SeasonsParsed|EpisodesParsed|     Length|MinLength|MaxLength| Status|Active|Table|Language|         dt_inclusao|
+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+-------+------+-----+--------+--------------------+
|      House of Cards|     Political drama|     political,drama| 1-Feb-13|6 seasons, 73 epi...|            6|            73| 42–59 min.|       42|       59|  Ended|     0|Drama| English|2021-03-16T21:20:...|
|       Hemlock Grove|     Horror/thriller|     horror,thriller|19-Apr-13|3 seasons, 33 epi...|            3|            33| 45–58 min.|       45|       58|  End

###### Exercícios

In [134]:
# 1. Transformar os campos "Premiere" e "dt_inclusao" de string para datetime.

## IMPORTANTE:
# Primeiro rodar o ex 6 que contem a funcao date_converter
df = df.withColumn('Premiere',mydate_converter(col('Premiere')))
df = df.withColumn("dt_inclusao",df.dt_inclusao.cast("date"))

df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: date (nullable = true)



In [135]:
df.show(10)

+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+-----------+---------+---------+-------+------+-----+--------+-----------+
|               Title|               Genre|         GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|     Length|MinLength|MaxLength| Status|Active|Table|Language|dt_inclusao|
+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+-----------+---------+---------+-------+------+-----+--------+-----------+
|      House of Cards|     Political drama|     political,drama|2013-02-01|6 seasons, 73 epi...|            6|            73| 42–59 min.|       42|       59|  Ended|     0|Drama| English| 2021-03-16|
|       Hemlock Grove|     Horror/thriller|     horror,thriller|2013-04-19|3 seasons, 33 epi...|            3|            33| 45–58 min.|       45|       58|  Ended|     0|Drama| English| 2021-03-16|


In [90]:
# 2. Ordenar os dados por ativos e gênero de forma decrescente, 0 = inativo e 1 = ativo, todos com número 1 devem aparecer primeiro.

df_temp = df.sort(col('Active').desc(),col('Genre').desc())
df_temp.show(20)

+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+--------+--------+-----------+
|               Title|               Genre|         GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|   Table|Language|dt_inclusao|
+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+--------+--------+-----------+
|          True Tunes|chrildrens musica...|chrildrens,musica...|2019-07-12|                 TBA|            0|             0|       TBA|        0|        0|Pending|     1|Children| English| 2021-03-16|
|Super Monsters Mo...|chrildrens musica...|chrildrens,musica...|2018-09-14|1 season, 4 episodes|            1|             4|    2 min.|        2|        2|Pending|     1|Children| English| 20

In [91]:
# 3. Remover linhas duplicadas e trocar o resultado das linhas que tiverem a coluna 
# "Seasons" de "TBA" para "a ser anunciado".

# 3.1 Remover duplicas
print(f'Qtde linhas : {df.count()}')
df_dist= df.dropDuplicates()
print(f'Qtde linhas distintas : {df_dist.count()}')

# 3.2 Trocando valor na coluna Seasons
df_dist = df_dist.withColumn('Seasons',regexp_replace('Seasons','TBA','a ser anunciado'))
temp = df_dist.select('*').filter('Seasons like "%a ser anunciado%"')
print (f'Qtde linhas coluna Season alterado : {temp.count()}')

Qtde linhas : 358
Qtde linhas distintas : 358
Qtde linhas coluna Season alterado : 15


In [144]:
# 4. Criar uma coluna nova chamada "Data de Alteração" e dentro dela um timestamp.

df2 = df_dist\
.withColumn("Data de Alteração",\
            when(df_dist.Seasons == "a ser anunciado",current_timestamp())\
            .otherwise(None))

df2.select('*').filter('Seasons like "%a ser anunciado%"').show()

+--------------------+--------------------+--------------------+----------+---------------+-------------+--------------+-------+---------+---------+-------+------+---------------+----------+-----------+--------------------+
|               Title|               Genre|         GenreLabels|  Premiere|        Seasons|SeasonsParsed|EpisodesParsed| Length|MinLength|MaxLength| Status|Active|          Table|  Language|dt_inclusao|   Data de Alteração|
+--------------------+--------------------+--------------------+----------+---------------+-------------+--------------+-------+---------+---------+-------+------+---------------+----------+-----------+--------------------+
|     Family Business|              Comedy|              comedy|2019-06-28|a ser anunciado|            0|             0|29 min.|        0|        0|Pending|     1|ForeignLanguage|    French| 2021-03-16|2022-03-13 22:59:...|
|  The Naked Director|               Drama|               drama|2019-08-08|a ser anunciado|            0

In [145]:
# 5. Trocar os nomes das colunas de inglês para português, exemplo: "Title" para "Título" (com acentuação).

newColumns = [ 'Título',
               'Gênero',
               'Rótulos de Gênero',
               'Pré Estreia',
               'Temporadas',
               'Temporadas Exibidas',
               'Episódios Exibidos',
               'Duração',
               'Duração Mínima',
               'Duração Máxima',
               'Status',
               'Ativo',
               'Rótulo',
               'Língua',
               'Data Inclusão',
               'Data de Alteração'
              ]

df2 = df2.toDF(*newColumns)
df2.printSchema()

root
 |-- Título: string (nullable = true)
 |-- Gênero: string (nullable = true)
 |-- Rótulos de Gênero: string (nullable = true)
 |-- Pré Estreia: date (nullable = true)
 |-- Temporadas: string (nullable = true)
 |-- Temporadas Exibidas: long (nullable = true)
 |-- Episódios Exibidos: long (nullable = true)
 |-- Duração: string (nullable = true)
 |-- Duração Mínima: long (nullable = true)
 |-- Duração Máxima: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Ativo: long (nullable = true)
 |-- Rótulo: string (nullable = true)
 |-- Língua: string (nullable = true)
 |-- Data Inclusão: date (nullable = true)
 |-- Data de Alteração: timestamp (nullable = true)



In [146]:
# 6. Testar e verificar se existe algum erro de processamento do spark e identificar onde pode ter ocorrido o erro.
# >>> Erro ocorrido em transformar a coluna 'Premiere' em date no ex 1, devido ao mes por extenso (1-Feb-13).

#funcao
def date_converter(x):
    if type(x) == str:
        d = datetime.strptime(x,'%d-%b-%y')
        str_date = d.strftime('%Y-%m-%d')
        return str_date
    else:
        return None

# User-Defined Functions (UDFs) são funções que executam cada linha 
mydate_converter = udf(date_converter)

In [147]:
# 7. Criar apenas 1 . csv com as seguintes colunas que foram nomeadas anteriormente 
# "Title,Genre, Seasons, Premiere, Language, Active, Status, dt_inclusao, Data de Alteração" as
# colunas devem estar em português com header e separadas por ";".

df_csv = df2.select('Título','Gênero','Temporadas','Pré Estreia','Língua','Ativo','Status','Data Inclusão','Data de Alteração')

# Spark df to Pandas df
df_csv2 = df_csv.toPandas()

df_csv2.to_csv("df_csv2.csv", sep = ';')


In [ ]:
# 8. Inserir esse . csv dentro de um bucket do AWS s3
# obs: estou com problema na verificação da minha conta AWS, por isso não foi possivel testar o insert abaixo no bucket 
# ou utilizar o databricks.


spark.sparkContext.hadoopConfiguration.set("fs.s3a.secret.key", "aws secretkey value")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.endpoint", "s3.amazonaws.com")

df_csv.write.option("header","true").csv("s3a://sparkbyexamples/csv/zipcodes")